In [ ]:
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage pandas


from google.cloud import bigquery
import pandas as pd
import numpy as np
from datetime import datetime, timezone


AttributeError: partially initialized module 'pandas' from 'h:\multiagent_human_worker\venv\Lib\site-packages\pandas\__init__.py' has no attribute 'core' (most likely due to a circular import)

In [41]:
# If Colab doesn't auto-detect your default project, set it explicitly:
PROJECT_ID = "rapid-airship-485407-c1"
client = bigquery.Client(project=PROJECT_ID)

In [8]:
table_id = "gdelt-bq.gdeltv2.gkg_partitioned"
table = client.get_table(table_id)
[(f.name, f.field_type) for f in table.schema]


[('GKGRECORDID', 'STRING'),
 ('DATE', 'INTEGER'),
 ('SourceCollectionIdentifier', 'INTEGER'),
 ('SourceCommonName', 'STRING'),
 ('DocumentIdentifier', 'STRING'),
 ('Counts', 'STRING'),
 ('V2Counts', 'STRING'),
 ('Themes', 'STRING'),
 ('V2Themes', 'STRING'),
 ('Locations', 'STRING'),
 ('V2Locations', 'STRING'),
 ('Persons', 'STRING'),
 ('V2Persons', 'STRING'),
 ('Organizations', 'STRING'),
 ('V2Organizations', 'STRING'),
 ('V2Tone', 'STRING'),
 ('Dates', 'STRING'),
 ('GCAM', 'STRING'),
 ('SharingImage', 'STRING'),
 ('RelatedImages', 'STRING'),
 ('SocialImageEmbeds', 'STRING'),
 ('SocialVideoEmbeds', 'STRING'),
 ('Quotations', 'STRING'),
 ('AllNames', 'STRING'),
 ('Amounts', 'STRING'),
 ('TranslationInfo', 'STRING'),
 ('Extras', 'STRING')]

In [6]:
timestamp_1 = 1627679830425
timestamp_2 = 1627938153438
timestamp_3 = 1628083724757
timestamp_4 = 1628722253112
today = 1769195945649
heldout_timestamp = 1629826110674

timestamps = [timestamp_1, timestamp_2, timestamp_3, timestamp_4, heldout_timestamp, today]

def unix_to_datatime(timestamp_ms):
    return datetime.fromtimestamp(timestamp_ms / 1000, tz=timezone.utc)


for time in timestamps:
    print(unix_to_datatime(time))


NameError: name 'datetime' is not defined

In [22]:
from google.cloud import bigquery
import numpy as np

def get_schwartz_vector(date_str, client):
    """
    Robust GDELT Query that handles sparse data and provides fallbacks.
    Targeting 2015-2023 window.
    """
    
    # We broaden the query to capture LIWC (c5) as a backup for Moral Foundations (c25)
    sql = """
    SELECT
      -- --- EGO VALUES (General Inquirer - Usually Reliable) ---
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c2.156:([0-9.]+)') AS FLOAT64)) as val_power,
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c5.7:([0-9.]+)') AS FLOAT64))   as val_achievement,
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c2.152:([0-9.]+)') AS FLOAT64)) as val_hedonism,
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c2.17:([0-9.]+)') AS FLOAT64))  as val_stimulation,

      -- --- SOCIAL VALUES (Primary: Moral Foundations c25) ---
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.4:([0-9.]+)') AS FLOAT64))  as val_universalism, 
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.6:([0-9.]+)') AS FLOAT64))  as val_benevolence,  
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.10:([0-9.]+)') AS FLOAT64)) as val_tradition,    
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.8:([0-9.]+)') AS FLOAT64))  as val_conformity,   
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.2:([0-9.]+)') AS FLOAT64))  as val_security,
      
      -- --- BACKUP SIGNALS (LIWC c1/c5 - Always Present) ---
      -- Used if c25 returns NULL
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c5.31:([0-9.]+)') AS FLOAT64))  as backup_social, -- Social processes
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c5.46:([0-9.]+)') AS FLOAT64))  as backup_risk,   -- Risk/Danger (Security)
      AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c5.10:([0-9.]+)') AS FLOAT64))  as backup_anger   -- Anger (Conformity/Tradition)

    FROM `gdelt-bq.gdeltv2.gkg_partitioned`
    -- 7-Day Window centered on target
    WHERE _PARTITIONDATE BETWEEN DATE_SUB(DATE(@target_date), INTERVAL 3 DAY) 
                             AND DATE_ADD(DATE(@target_date), INTERVAL 3 DAY)
      AND V2Locations LIKE '%#US#%'
      AND GCAM IS NOT NULL
    """
    
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_date", "DATE", date_str)
        ]
    )
    
    try:
        results = list(client.query(sql, job_config=job_config).result())
        if not results:
            # Fatal error: No GDELT data at all for this week (Rare)
            return get_flat_prior()
        row = results[0]
    except Exception as e:
        print(f"Query Failed: {e}")
        return get_flat_prior()

    # --- DEFENSIVE DATA MAPPING ---
    
    # Helper: "Zero if None"
    def z(val): return val if val is not None else 0.0

    # 1. Ego Values (Direct Map)
    scores = {
        "POWER":       z(row.val_power),
        "ACHIEVEMENT": z(row.val_achievement),
        "HEDONISM":    z(row.val_hedonism),
        "STIMULATION": z(row.val_stimulation)
    }

    # 2. Social Values (With Fallback Logic)
    # Check if MFD is missing (i.e., Security is 0.0)
    mfd_present = (z(row.val_security) + z(row.val_universalism)) > 0.001

    if mfd_present:
        # Use High-Quality MFD signals
        scores["UNIVERSALISM"] = z(row.val_universalism)
        scores["BENEVOLENCE"]  = z(row.val_benevolence)
        scores["TRADITION"]    = z(row.val_tradition)
        scores["CONFORMITY"]   = z(row.val_conformity)
        scores["SECURITY"]     = z(row.val_security)
    else:
        # FALLBACK: Use LIWC proxies if MFD is broken (Common in 2015)
        # print("Warning: Using LIWC Fallback for date", date_str)
        scores["UNIVERSALISM"] = z(row.backup_social) * 0.5
        scores["BENEVOLENCE"]  = z(row.backup_social) * 0.5
        scores["TRADITION"]    = z(row.backup_anger) * 0.5
        scores["CONFORMITY"]   = z(row.backup_anger) * 0.8
        scores["SECURITY"]     = z(row.backup_risk)

    # 3. Normalization (Softmax or Standard Ratio)
    # Standard Ratio is safer for "Prompt Injection" readability
    total = sum(scores.values())
    
    if total == 0:
        return get_flat_prior()
        
    return {k: round(v/total, 3) for k, v in scores.items()}

def get_flat_prior():
    """Returns equal probability if data fails completely"""
    return {
        "POWER": 0.11, "ACHIEVEMENT": 0.11, "HEDONISM": 0.11,
        "STIMULATION": 0.11, "UNIVERSALISM": 0.11, "BENEVOLENCE": 0.11,
        "TRADITION": 0.11, "CONFORMITY": 0.11, "SECURITY": 0.11
    }

In [23]:
unix_to_datatime(heldout_timestamp)
date_str = unix_to_datatime(heldout_timestamp).strftime("%Y-%m-%d")
date_str ="2015-07-15"
print(f"Querying GDELT for date: {date_str}")
gdelts_vector = get_schwartz_vector(date_str, client)
print("GDELT Schwartz Value Vector:")
print(gdelts_vector)

Querying GDELT for date: 2015-07-15
GDELT Schwartz Value Vector:
{'POWER': 0.136, 'ACHIEVEMENT': 0.083, 'HEDONISM': 0.029, 'STIMULATION': 0.034, 'UNIVERSALISM': 0.012, 'BENEVOLENCE': 0.012, 'TRADITION': 0.109, 'CONFORMITY': 0.174, 'SECURITY': 0.411}


In [25]:
#open traininggdelt.jsonl, val, test and extract all timestamps for a user_id
import json
#{"user_id": "AE3KLVXGZPANXE5XLXYKHTVAZ3FQ", "category": "All_Beauty", "history": [{"parent_asin": "B095RWJJB8", "rating": 4.0, "timestamp_ms": 1627679830425, "review_excerpt": "This is a pretty bow however $7 for one bow is pretty expensive considering I can get 10 of these bows for $8 from other sellers.", "review_full": "This is a pretty bow however $7 for one bow is pretty expensive considering I can get 10 of these bows for $8 from other sellers.", "review_title": "Pretty but overpriced", "product": {"title": "Summer Crystal Hair Clip Sparkling Sequins, Double-Layered Alligator Clip Hair Bow Accessory For Women and Girls, Made in Korea, Daily, Party, Cosplay (Holographic)", "brand": null, "price": null, "main_category": "All Beauty"}}, {"parent_asin": "B097JXPZ6D", "rating": 4.0, "timestamp_ms": 1627938153438, "review_excerpt": "This is a cute bow and is exactly what is advertised. I do believe the $10 price point is pretty high considering you can get 10 headbands for $12. It is well made and fits my 4 year old daughter\u2019s head nicely.", "review_full": "This is a cute bow and is exactly what is advertised. I do believe the $10 price point is pretty high considering you can get 10 headbands for $12. It is well made and fits my 4 year old daughter\u2019s head nicely.", "review_title": "Pretty headband", "product": {"title": "Summer Crystal Headband for Girls, 3D Large Glitter Top Bow, Hair Accessory for Girls and Women, Various Occasions, Holidays, Parties, Daily, Cosplay, Gift (Magenta)", "brand": null, "price": null, "main_category": "All Beauty"}}, {"parent_asin": "B08Q8NQMX2", "rating": 4.0, "timestamp_ms": 1628083724757, "review_excerpt": "These are cute and my 4 year old daughter loves them. They come in bright colors however a handful do them has creases wings and I\u2019m not really sure how to get the crease out.", "review_full": "These are cute and my 4 year old daughter loves them. They come in bright colors however a handful do them has creases wings and I\u2019m not really sure how to get the crease out.", "review_title": "Cute butterfly clips but some wings are creased", "product": {"title": "DARKLATER Butterfly Hair Clips for Girls,for Toddler Girls,Baby Girls and Women,Cute Hair Clips,Beautiful Hair Accessories,12 PCS", "brand": null, "price": null, "main_category": "All Beauty"}}, {"parent_asin": "B093JGCRWX", "rating": 3.0, "timestamp_ms": 1628722253112, "review_excerpt": "If this product was indeed EWG verified, it would not only be on the website but it would have the EWG logo on the product plus it wouldn\u2019t have linalool which is high on the allergy list.<br /><br />Other than the linalool, this has decent ingredients. I would stay away from this product if you have malassezia (fungal) acne as olive and japonica may be triggers and/or pore clogging.<br /><br />Like all natural bar shampoos, it won\u2019t lather like traditional synthetic shampoos but it does clean. It takes some getting use too and a period of detoxing for your hair to get use to the change in chemicals if you are switching from synthetic to natural but it is worth it!<br /><br />I would recommend this shampoo bar however I am rather concerned about the EWG verified claim.", "review_full": "I searched the EWG website for this company and product and in many spelling varieties and came up empty handed. If this product was indeed EWG verified, it would not only be on the website but it would have the EWG logo on the product plus it wouldn\u2019t have linalool which is high on the allergy list.<br /><br />Other than the linalool, this has decent ingredients. It is silicone free, paraben free, sulfate free and alcohol free. I would stay away from this product if you have malassezia (fungal) acne as olive and japonica may be triggers and/or pore clogging.<br /><br />Like all natural bar shampoos, it won\u2019t lather like traditional synthetic shampoos but it does clean. It takes some getting use too and a period of detoxing for your hair to get use to the change in chemicals if you are switching from synthetic to natural but it is worth it!<br /><br />I would recommend this shampoo bar however I am rather concerned about the EWG verified claim.", "review_title": "Paraben free, silicone free, sulfate free but not EWG verified", "product": {"title": "The Vegan Glow Quinoa Protein Shampoo Bar | EWG Verified | Vegetable proteins from Quinoa & Soybeans", "brand": null, "price": null, "main_category": "All Beauty"}}], "heldout": {"parent_asin": "B08Z7FQGW3", "rating": 4.0, "timestamp_ms": 1629826110674, "review_excerpt": "This is a beautiful dark purple leaf crown with rose gold metal. It fits my female adult head nicely and that was after I bent it to make it smaller. It wouldn\u2019t fit a small child. My 4 year old daughter was very disappointed that it didn\u2019t fit her. It came quickly and I\u2019m surprised it was damaged due to the lack of product protection. It is well made and a fun addition to anyone's dress up collection!", "review_full": "This is a beautiful dark purple leaf crown with rose gold metal. It fits my female adult head nicely and that was after I bent it to make it smaller. It wouldn\u2019t fit a small child. My 4 year old daughter was very disappointed that it didn\u2019t fit her. It came quickly and I\u2019m surprised it was damaged due to the lack of product protection. It is well made and a fun addition to anyone's dress up collection!", "review_title": "Beautiful crown for adults", "product": {"title": "S SNUOY Purple Crystal Vintage Queen Crowns Baroque Tiaras Wedding Bridal Queen Tiaras and Crowns for Women and Girls Party Headbands", "brand": null, "price": null, "main_category": "All Beauty"}}}
def extract_user_timestamps(filename, target_user_id):
    timestamps = []
    with open(filename, "r") as f:
        for line in f:
            data = json.loads(line)
            if data["user_id"] == target_user_id:
                for review in data["history"]:
                    timestamps.append(review["timestamp_ms"])
                # Also include heldout timestamp
                timestamps.append(data["heldout"]["timestamp_ms"])
                break
    return timestamps


In [34]:
user = extract_user_timestamps("traininggdelt.jsonl", "AECQQBG6YRYCOJL2NCB2H3V6LD6Q")
print(user)
#output: [1627679830425, 1627938153438, 1628083724757, 1628722253112, 1629826110674]
for time in [1627679830425, 1627938153438, 1628083724757, 1628722253112, 1629826110674]:
    print(unix_to_datatime(time))

[1624564949327, 1625872089176, 1629158373893, 1648740545679, 1666137829785]
2021-07-30 21:17:10.425000+00:00
2021-08-02 21:02:33.438000+00:00
2021-08-04 13:28:44.757000+00:00
2021-08-11 22:50:53.112000+00:00
2021-08-24 17:28:30.674000+00:00


In [ ]:
#open traininggdelt.jsonl, val, test and extract all timestamps for a user_id

with open("traininggdelt.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        print(data["user_id"])
        #add array to hold timestamps
        timestamps = []
        for review in data["history"]:
            timestamps.append(review["timestamp_ms"])
        #also add heldout timestamp
        timestamps.append(data["heldout"]["timestamp_ms"])

        
        print(timestamps)

AE3KLVXGZPANXE5XLXYKHTVAZ3FQ
[1627679830425, 1627938153438, 1628083724757, 1628722253112, 1629826110674]
AE3PLZHW6NXWBMZ76TDVFQG2MJFA
[1658088924308, 1658671602427, 1663849782220, 1669914635772, 1671214216176]
AEAXAJACFMXIAAH4WOHRMXPSZWFA
[1642973121851, 1644020283932, 1647202368573, 1647542575282, 1650576787877]
AECQQBG6YRYCOJL2NCB2H3V6LD6Q
[1624564949327, 1625872089176, 1629158373893, 1648740545679, 1666137829785]
AECTRGMRKOGAYIV3YXX73CQEQCSQ
[1613753146957, 1620942843347, 1624316551583, 1644173335551, 1663673403815]
AEFU6XKGH6J4ZIYTVVBGEDUCTZHQ
[1623704532061, 1632287090360, 1637089891493, 1642400709903, 1668052318532]
AEGKJTQWHGBIOTO34OB3GKIGCN7A
[1619011772443, 1625146282820, 1625587589330, 1631642711343, 1635873265775]
AEHZBUFEOW4TY3K4IYFFR5ZU5B3Q
[1630362683688, 1643531946332, 1651628189585, 1659463150371, 1665097462120]
AEJU3Z6HDAERETMYI2CXBQVPPDFA
[1674993110944, 1674993414313, 1675729769788, 1678928691080, 1682336195381]
AELHYMCYVS6T35HA3UH5UGFWTK6Q
[1634907324309, 1637956712

In [35]:
from google.cloud import bigquery
import pandas as pd
import datetime

def batch_process_users_with_window(user_data_list, client):
    """
    1. Identifies all necessary dates (target +/- 3 days).
    2. Queries GDELT once for that superset of dates.
    3. reconstructs the 7-day average for each user locally.
    """
    
    # --- STEP 1: Calculate the "Superset" of dates needed ---
    all_needed_dates = set()
    
    for user in user_data_list:
        target = pd.to_datetime(user['date'])
        # Add target and its neighbors to the set
        for i in range(-3, 4): # -3 to +3
            d = target + pd.Timedelta(days=i)
            all_needed_dates.add(d.strftime('%Y-%m-%d'))
            
    print(f"User Count: {len(user_data_list)}")
    print(f"Unique Daily Partitions to Scan: {len(all_needed_dates)}")

    # Format for SQL
    date_sql_string = ", ".join([f"DATE('{d}')" for d in all_needed_dates])

    # --- STEP 2: The Batch Query (Fetches Daily Granularity) ---
    sql = f"""
    SELECT
        _PARTITIONDATE as day,
        -- EGO VALUES
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c2.156:([0-9.]+)') AS FLOAT64)) as val_power,
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c5.7:([0-9.]+)') AS FLOAT64))   as val_achievement,
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c2.152:([0-9.]+)') AS FLOAT64)) as val_hedonism,
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c2.17:([0-9.]+)') AS FLOAT64))  as val_stimulation,
        
        -- SOCIAL VALUES
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.4:([0-9.]+)') AS FLOAT64))  as val_universalism, 
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.6:([0-9.]+)') AS FLOAT64))  as val_benevolence,  
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.10:([0-9.]+)') AS FLOAT64)) as val_tradition,    
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.8:([0-9.]+)') AS FLOAT64))  as val_conformity,   
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c25.2:([0-9.]+)') AS FLOAT64))  as val_security,

        -- BACKUP (LIWC)
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c5.31:([0-9.]+)') AS FLOAT64))  as backup_social,
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c5.46:([0-9.]+)') AS FLOAT64))  as backup_risk,
        AVG(SAFE_CAST(REGEXP_EXTRACT(GCAM, r'c5.10:([0-9.]+)') AS FLOAT64))  as backup_anger

    FROM `gdelt-bq.gdeltv2.gkg_partitioned`
    WHERE _PARTITIONDATE IN ({date_sql_string})
      AND V2Locations LIKE '%#US#%'
      AND GCAM IS NOT NULL
    GROUP BY day
    """
    
    # Download everything to a Pandas DataFrame
    # Note: The 'day' column will be type 'db_date', convert to datetime64
    df_daily = client.query(sql).to_dataframe()
    df_daily['day'] = pd.to_datetime(df_daily['day'])
    
    # Index by date for O(1) lookups
    df_daily = df_daily.set_index('day')

    # --- STEP 3: The Reconstruction Loop (Python Side) ---
    final_output = []
    
    for user in user_data_list:
        target_date = pd.to_datetime(user['date'])
        start_win = target_date - pd.Timedelta(days=3)
        end_win = target_date + pd.Timedelta(days=3)
        
        # Slice the dataframe for this user's window
        # .loc[] includes the end index, so this grabs the 7 rows we need
        # We use reindex() to handle missing days (e.g. if GDELT missed a day) gracefully
        window_dates = pd.date_range(start=start_win, end=end_win)
        user_window = df_daily.reindex(window_dates) 
        
        # Calculate the Mean of the 7-day window (ignoring NaNs)
        avg_row = user_window.mean(skipna=True)
        
        # Check if we have valid data (if all days were NaN, avg_row is all NaN)
        if pd.isna(avg_row['val_power']):
            # Complete failure for this week -> Flat Prior
            user['psych_vector'] = {k: 0.11 for k in ["POWER", "SECURITY", "HEDONISM"]} # abbreviated
        else:
            # --- Apply Normalization/Fallback Logic on the AVERAGED Row ---
            def z(x): return 0.0 if pd.isna(x) else float(x)

            scores = {
                "POWER":       z(avg_row['val_power']),
                "ACHIEVEMENT": z(avg_row['val_achievement']),
                "HEDONISM":    z(avg_row['val_hedonism']),
                "STIMULATION": z(avg_row['val_stimulation'])
            }

            # Fallback Check (Did the Moral Foundations dictionary run this week?)
            if (z(avg_row['val_security']) + z(avg_row['val_universalism'])) > 0.001:
                scores.update({
                    "UNIVERSALISM": z(avg_row['val_universalism']), "BENEVOLENCE": z(avg_row['val_benevolence']),
                    "TRADITION": z(avg_row['val_tradition']), "CONFORMITY": z(avg_row['val_conformity']),
                    "SECURITY": z(avg_row['val_security'])
                })
            else:
                # LIWC Fallback
                scores.update({
                    "UNIVERSALISM": z(avg_row['backup_social'])*0.5, "BENEVOLENCE": z(avg_row['backup_social'])*0.5,
                    "TRADITION": z(avg_row['backup_anger'])*0.5, "CONFORMITY": z(avg_row['backup_anger'])*0.8,
                    "SECURITY": z(avg_row['backup_risk'])
                })

            # Final Normalize
            total = sum(scores.values())
            vector = {k: round(v/total, 3) for k,v in scores.items()} if total > 0 else None
            user['psych_vector'] = vector

        final_output.append(user)
        
    return final_output

In [3]:
import json
from google.cloud import bigquery

# Your existing function
# def batch_process_users_with_window(user_data_list, client): ...

def enrich_files_with_gdelt(files, client):
    """
    Read all users, batch query GDELT, write back with psych_vector.
    """
    
    # Step 1: Read all users, track which file they came from
    all_users = []
    file_boundaries = []  # [(filename, start_idx, end_idx), ...]
    
    for filename in files:
        start_idx = len(all_users)
        with open(filename, "r") as f:
            for line in f:
                if not line.strip():
                    continue
                user = json.loads(line)
                # Rename selection_month -> date for the function
                user['date'] = user['selection_month']
                all_users.append(user)
        end_idx = len(all_users)
        file_boundaries.append((filename, start_idx, end_idx))
        print(f"Loaded {end_idx - start_idx} users from {filename}")
    
    print(f"\nTotal users to process: {len(all_users)}")
    
    # Step 2: Batch process all users (single GDELT query!)
    enriched_users = batch_process_users_with_window(all_users, client)
    
    # Step 3: Write back to files
    for filename, start_idx, end_idx in file_boundaries:
        users_for_file = enriched_users[start_idx:end_idx]
        
        # Clean up the temporary 'date' key we added
        for user in users_for_file:
            if 'date' in user:
                del user['date']
        
        out_filename = filename.replace(".jsonl", "_enriched.jsonl")
        with open(out_filename, "w") as f:
            for user in users_for_file:
                f.write(json.dumps(user) + "\n")
        print(f"Wrote {len(users_for_file)} users to {out_filename}")

# Run it
files = ["traininggdelt.jsonl", "validationgdelt.jsonl", "testgdelt.jsonl"]
enrich_files_with_gdelt(files, client)


KeyboardInterrupt: 

In [12]:
def fix_selection_dates(files):
    """
    For each user, find the actual review date in their selection month
    and replace the generic '15' with the real day.
    """
    
    for filename in files:
        updated_users = []
        fixed_count = 0
        
        with open(filename, "r") as f:
            for line in f:
                if not line.strip():
                    continue
                user = json.loads(line)
                
                # Parse selection_month: "2022-04-15" -> year=2022, month=4
                selection = user.get('selection_month', '')
                if not selection:
                    updated_users.append(user)
                    continue
                
                sel_year = int(selection[:4])
                sel_month = int(selection[5:7])
                
                # Collect all timestamps from history + heldout
                all_timestamps = []
                for review in user.get('history', []):
                    ts = review.get('timestamp_ms')
                    if ts:
                        all_timestamps.append(ts)
                heldout_ts = user.get('heldout', {}).get('timestamp_ms')
                if heldout_ts:
                    all_timestamps.append(heldout_ts)
                
                # Find first timestamp that matches the selection year/month
                new_date = None
                for ts in sorted(all_timestamps):
                    dt = datetime.utcfromtimestamp(ts / 1000)
                    if dt.year == sel_year and dt.month == sel_month:
                        new_date = dt.strftime('%Y-%m-%d')
                        break
                
                if new_date and new_date != selection:
                    user['selection_month'] = new_date
                    fixed_count += 1
                
                updated_users.append(user)
        
        # Write back
        with open(filename, "w") as f:
            for user in updated_users:
                f.write(json.dumps(user) + "\n")
        
        print(f"{filename}: Fixed {fixed_count}/{len(updated_users)} dates")


In [14]:

from datetime import datetime, timezone

files = ["traininggdelt.jsonl", "validationgdelt.jsonl", "testgdelt.jsonl", "temp.jsonl "]
fix_selection_dates(files)

traininggdelt.jsonl: Fixed 0/49 dates
validationgdelt.jsonl: Fixed 0/40 dates
testgdelt.jsonl: Fixed 0/43 dates
temp.jsonl : Fixed 0/283 dates


C:\Users\yashm\AppData\Local\Temp\ipykernel_19352\3982858135.py:39: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(ts / 1000)


In [ ]:
def calibrate_psych_vector(raw_vector):
    """
    Balances the Schwartz Vector by penalizing 'loud' dictionaries (General Inquirer)
    and boosting 'quiet' dictionaries (Moral Foundations).
    """
    
    # 1. Define Multipliers (derived from GDELT base rates)
    # Power/Achievement are ~4x louder than average -> Multiply by 0.25
    # Security/Conformity are ~3x quieter than average -> Multiply by 3.0
    
    weights = {
        # --- LOUD SIGNALS (Turn Down) ---
        "POWER":        0.25, 
        "ACHIEVEMENT":  0.30,
        "HEDONISM":     0.50,  # Moderately loud
        "STIMULATION":  0.50,
        
        # --- QUIET SIGNALS (Turn Up) ---
        "SECURITY":     3.0,
        "CONFORMITY":   2.5,
        "TRADITION":    2.0,
        "BENEVOLENCE":  2.0,
        "UNIVERSALISM": 2.5
    }
    
    # 2. Apply Weights
    weighted_scores = {}
    for trait, score in raw_vector.items():
        w = weights.get(trait, 1.0)
        weighted_scores[trait] = score * w
        
    # 3. Re-Normalize to sum to 1.0
    total = sum(weighted_scores.values())
    
    # Safety check
    if total == 0: return raw_vector 
    
    normalized_vector = {k: round(v/total, 3) for k, v in weighted_scores.items()}
    
    return normalized_vector

# --- EXAMPLE TEST ---
raw = {
    "POWER": 0.424, "ACHIEVEMENT": 0.267, "HEDONISM": 0.074, 
    "STIMULATION": 0.095, "UNIVERSALISM": 0.02, "BENEVOLENCE": 0.027, 
    "TRADITION": 0.031, "CONFORMITY": 0.023, "SECURITY": 0.039
}

calibrated = calibrate_psych_vector(raw)
print("Original POWER:", raw['POWER'])      # 0.424 (Dominant)
print("Fixed POWER:   ", calibrated['POWER']) # ~0.14 (Balanced)
print("Fixed SECURITY:", calibrated['SECURITY']) # ~0.16 (Now meaningful)